In [2]:
import json
import os

In [12]:
import json

def generate_markdown_table(logic_lm_json, direct_json, cot_json):
    """Generates markdown table from JSON data.
    """

    table_header = "| Dataset | gemini-1.0-pro-vision-001 | gemini-1.5-pro-preview-0409 | gemini-1.5-pro-preview-0514 | gemini-1.5-flash-preview-0514 |\n"
    table_header += "|---|---|---|---|---| \n"

    table_rows = []
    for task in logic_lm_json["gemini-1.0-pro-vision-001"]:
        logic_lm_score = logic_lm_json["gemini-1.0-pro-vision-001"][task]["Overall_Accuracy"]
        direct_score = direct_json["gemini-1.5-flash-preview-0514"][task]["Direct"]["Average_EM_score"]
        cot_score = cot_json["gemini-1.5-flash-preview-0514"][task]["CoT"]["Average_EM_score"]
        gemini_1_5_pro_preview_0409_score = direct_json["gemini-1.5-pro-preview-0409"][task]["Direct"]["Average_EM_score"]
        gemini_1_5_pro_preview_0514_score = direct_json["gemini-1.5-pro-preview-0514"][task]["Direct"]["Average_EM_score"]
        table_rows.append(f"| {task} | {logic_lm_score:.2f} | {gemini_1_5_pro_preview_0409_score:.2f} | {gemini_1_5_pro_preview_0514_score:.2f} | {direct_score:.2f} | {cot_score:.2f} |")

    table_string = table_header + "\n".join(table_rows)
    return table_string

# Load the JSON data
with open("./baselines/evaluation/evaluation_baselines.json", "r") as f:
    baseline_results = json.load(f)

with open("./outputs/evaluation/evaluation_logic_programs.json", "r") as f:
    logic_lm_results = json.load(f)

# Generate markdown table
markdown_table = generate_markdown_table(baseline_results, logic_lm_results)

# Print markdown table
print(markdown_table)

FileNotFoundError: [Errno 2] No such file or directory: 'logic_lm_data.json'

In [11]:
import json

def generate_markdown_table(data):
    """Generates a markdown table from a JSON dictionary.

    Args:
        data: A dictionary containing the data for the table.

    Returns:
        A string containing the markdown table.
    """

    # Get the model names from the first level of the dictionary
    models = list(data.keys())

    # Get the task names from the second level of the dictionary
    tasks = list(data[models[0]].keys())

    # Create the table header
    table = "| Dataset | " + " | ".join(models) + " | \n"
    table += "|---|---|" + "---|" * (len(models) - 1) + "|\n"

    # Add the data to the table
    for task in tasks:
        table += f"| {task} | "
        for model in models:
            table += f"{data[model][task]:.2f} | "
        table += "\n"

    return table

# Example usage:
data = {
    "ChatGPT": {
        "ProntoQA": 47.40,
        "ProofWriter": 35.50,
        "FOLIO": 45.09,
        "LogicalDeduction": 40.00,
        "AR-LSAT": 20.34
    },
    "GPT-3.5": {
        "ProntoQA": 51.80,
        "ProofWriter": 36.16,
        "FOLIO": 54.60,
        "LogicalDeduction": 41.33,
        "AR-LSAT": 22.51
    },
    "GPT-4": {
        "ProntoQA": 77.40,
        "ProofWriter": 52.67,
        "FOLIO": 69.11,
        "LogicalDeduction": 71.33,
        "AR-LSAT": 33.33
    }
}

markdown_table = generate_markdown_table(data)
print(markdown_table)

| Dataset | ChatGPT | GPT-3.5 | GPT-4 | 
|---|---|---|---||
| ProntoQA | 47.40 | 51.80 | 77.40 | 
| ProofWriter | 35.50 | 36.16 | 52.67 | 
| FOLIO | 45.09 | 54.60 | 69.11 | 
| LogicalDeduction | 40.00 | 41.33 | 71.33 | 
| AR-LSAT | 20.34 | 22.51 | 33.33 | 



In [10]:
def generate_markdown_table(data):
    """Generates a markdown table from a JSON dictionary.

    Args:
        data: A dictionary containing the data for the table.

    Returns:
        A string containing the markdown table.
    """

    # Get the headers from the first row of the data
    headers = list(data[0].keys())

    # Create the table header row
    table = "| " + " | ".join(headers) + " |" + "\n"
    table += "|-" * (len(headers) + 1) + "\n"

    # Add the data rows
    for row in data:
        table += "| " + " | ".join([str(value) for value in row.values()]) + " |" + "\n"

    return table

def generate_github_markdown(data, filename="table.md"):
    """Generates a GitHub markdown file from a JSON dictionary.

    Args:
        data: A dictionary containing the data for the table.
        filename: The name of the output markdown file.
    """

    # Generate the markdown table
    markdown_table = generate_markdown_table(data)

    # Write the markdown table to a file
    with open(filename, "w") as f:
        f.write(markdown_table)

# Example JSON data
data = [
    {"Dataset": "ProntoQA", "Standard": 47.40, "CoT": 67.80, "Logic-LM": 61.00, "Standard": 51.80, "CoT": 83.00, "Logic-LM": 85.00, "Standard": 77.40, "CoT": 98.79, "Logic-LM": 83.20},
    {"Dataset": "ProofWriter", "Standard": 35.50, "CoT": 49.17, "Logic-LM": 58.33, "Standard": 36.16, "CoT": 48.33, "Logic-LM": 71.45, "Standard": 52.67, "CoT": 68.11, "Logic-LM": 79.66},
    {"Dataset": "FOLIO", "Standard": 45.09, "CoT": 57.35, "Logic-LM": 62.74, "Standard": 54.60, "CoT": 57.84, "Logic-LM": 61.27, "Standard": 69.11, "CoT": 70.58, "Logic-LM": 78.92},
    {"Dataset": "LogicalDeduction", "Standard": 40.00, "CoT": 42.33, "Logic-LM": 65.67, "Standard": 41.33, "CoT": 48.33, "Logic-LM": 62.00, "Standard": 71.33, "CoT": 75.25, "Logic-LM": 87.63},
    {"Dataset": "AR-LSAT", "Standard": 20.34, "CoT": 17.31, "Logic-LM": 26.41, "Standard": 22.51, "CoT": 22.51, "Logic-LM": 25.54, "Standard": 33.33, "CoT": 35.06, "Logic-LM": 43.04}
]

# # Generate the GitHub markdown file
# generate_github_markdown(data)

# # Load the JSON data
# with open("./baselines/evaluation/evaluation_baselines.json", "r") as f:
#     baseline_results = json.load(f)

# with open("./outputs/evaluation/evaluation_logic_programs.json", "r") as f:
#     logic_lm_results = json.load(f)

# Generate the markdown table
markdown_table = generate_markdown_table(data)

# Print the markdown table
print(markdown_table)

| Dataset | Standard | CoT | Logic-LM |
|-|-|-|-|-
| ProntoQA | 77.4 | 98.79 | 83.2 |
| ProofWriter | 52.67 | 68.11 | 79.66 |
| FOLIO | 69.11 | 70.58 | 78.92 |
| LogicalDeduction | 71.33 | 75.25 | 87.63 |
| AR-LSAT | 33.33 | 35.06 | 43.04 |



In [2]:
gemini_models = ["gemini-1.0-pro-vision-001", "gemini-1.5-pro-preview-0409"]

In [6]:
gemini_models = ['gemini-1.5-flash-preview-0514']

In [7]:
dataset_name = "AR-LSAT"
split = "dev"
model_name = gemini_models[0]
backup = "random"

program_path = os.path.join("outputs/logic_programs", f"{dataset_name}_{split}_{model_name}.json")
result_path = os.path.join("outputs/logic_inference", f"{dataset_name}_{split}_{model_name}_backup-{backup}.json")

with open(program_path, "r") as json_file:
    programs = json.load(json_file)

with open(result_path, "r") as json_file:
    results = json.load(json_file)

In [1]:
dataset_name = "AR-LSAT"
split = "dev"
model_name = "gpt-4"
backup = "random"

program_path = os.path.join("outputs/logic_programs", f"{dataset_name}_{split}_{model_name}.json")
result_path = os.path.join("outputs/logic_inference", f"{dataset_name}_{split}_{model_name}_backup-{backup}.json")

with open(program_path, "r") as json_file:
    programs_gpt = json.load(json_file)

with open(result_path, "r") as json_file:
    results_gpt = json.load(json_file)


NameError: name 'json' is not defined

In [6]:
for program, result, program_gpt, result_gpt in zip(programs, results, programs_gpt, results_gpt):
    if result_gpt["flag"] == "success" and result["flag"] in ["parsing error", "execution error"]:
        print(program["raw_logic_programs"][0])
        break

In [7]:
for program, result in zip(programs, results):
    # if result["id"] ==  "ar_lsat_200006_1-G_1_6":
    #     print(result["flag"])
    # print(result["flag"])
    if result["flag"] == "success":
        print(result["flag"])
        print(program["raw_logic_programs"][0])
        break


In [3]:
from models.symbolic_solvers.z3_solver.sat_problem_solver import LSAT_Z3_Program

In [12]:
logic_program = '''# Declarations
people = EnumSort([Vladimir, Wendy])
meals = EnumSort([breakfast, lunch, dinner, snack])
foods = EnumSort([fish, hot_cakes, macaroni, omelet, poached_eggs])
eats = Function([people, meals] -> [foods])

# Constraints
ForAll([m:meals], eats(Vladimir, m) != eats(Wendy, m)) ::: At no meal does Vladimir eat the same kind of food as Wendy
ForAll([p:people, f:foods], Count([m:meals], eats(p, m) == f) <= 1) ::: Neither of them eats the same kind of food more than once during the day
ForAll([p:people], Or(eats(p, breakfast) == hot_cakes, eats(p, breakfast) == poached_eggs, eats(p, breakfast) == omelet)) ::: For breakfast, each eats exactly one of the following: hot cakes, poached eggs, or omelet
ForAll([p:people], Or(eats(p, lunch) == fish, eats(p, lunch) == hot_cakes, eats(p, lunch) == macaroni, eats(p, lunch) == omelet)) ::: For lunch, each eats exactly one of the following: fish, hot cakes, macaroni, or omelet
ForAll([p:people], Or(eats(p, dinner) == fish, eats(p, dinner) == hot_cakes, eats(p, dinner) == macaroni, eats(p, dinner) == omelet)) ::: For dinner, each eats exactly one of the following: fish, hot cakes, macaroni, or omelet
ForAll([p:people], Or(eats(p, snack) == fish, eats(p, snack) == omelet)) ::: For a snack, each eats exactly one of the following: fish or omelet
eats(Wendy, lunch) == omelet ::: Wendy eats an omelet for lunch

# Options
Question ::: Vladimir must eat which one of the following foods?
is_valid(Exists([m:meals], eats(Vladimir, m) == fish)) ::: (A)
is_valid(Exists([m:meals], eats(Vladimir, m) == hot_cakes)) ::: (B)
is_valid(Exists([m:meals], eats(Vladimir, m) == macaroni)) ::: (C)
is_valid(Exists([m:meals], eats(Vladimir, m) == omelet)) ::: (D)
is_valid(Exists([m:meals], eats(Vladimir, m) == poached_eggs)) ::: (E)'''


z3_program = LSAT_Z3_Program(logic_program, 'AR-LSAT')
# print(z3_program.standard_code)

output, error_message = z3_program.execute_program()
print(output)
# print(z3_program.answer_mapping(output))

['(D)']
